# TF指南

本文的目的是向读者展示如何利用TF在MNIST数据集上训练并评估一个用于识别手写数字的简易前馈神经网络（Feed-Forward NN），适合有兴趣使用TF的资深机器学习者。

因此，本文不会向大家介绍机器学习领域的基础知识，并假定读者已经按装好TF必须的环境。

## 涉及的文件

- [mnist.py](https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/examples/tutorials/mnist/mnist.py)
  - 构建一个完全连接的MNIST模型所需的代码
- [fully_connected_feed.py](https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/examples/tutorials/mnist/fully_connected_feed.py)
  - 利用下载的数据集训练构建好的MNIST模型的主要代码，以数据反馈字典的形式作为输入，只需要运行这个文件即可启动训练

## 准备数据

仍然是MNIST数据集，一些列28*28像素的单个手写数字灰度图片，通过训练模型来识别类似图片中的内容是0-9的哪一个。

更多信息，参阅[Yann Lecun官网关于MNIST的介绍](http://yann.lecun.com/exdb/mnist/)或者[Chris Olah对MNIST可视化探索](http://colah.github.io/posts/2014-10-Visualizing-MNIST/)。

![](./图表/MNIST.png)

### 下载

在run_training()方法时，input_data.read_data_sets()函数会确保本地训练文件夹下已正确下载了MNIST数据集，然后把这些数据解压并返回一个含有DataSet实例的字典。

    data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

注意：fake_data标记用于单元测试，读者可以不理会。

- data_sets.train
  - 55000个图片和标签，作为主要训练集
- data_sets.validation
  - 5000个图片和标签，用于迭代验证训练准确度
- data_sets.test
  - 10000个图片和标签，用于最终测试训练准确度

## 输入与占位符

    placeholder_inputs()函数将生成两个tf.placeholder操作，定义传入计算图中的shape参数，shape参数包括batch_size值，后续还会将实际的训练用例传入计算图。

    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

在训练迭代的后续步骤中，传入的整个图片和标签数据集会被切片，以符合每一个操作所设置的batch_size值，占位符操作会填补符合这个batch_size值；然后使用feed_dict参数，将数据传入sess.run()函数。

## 构建计算图

在为数据创建好占位符之后，就可以运行mnist.py文件，经过三个阶段的模式函数操作，计算图就构建完成：

- inference()
  - 尽可能地构建好计算图，满足促使神经网络向前反馈并作出预测的要求
- loss()
  - 往inference中添加生成损失所需的操作
- training()
  - 往损失中添加计算兵应用梯度下降所需的操作

![](./图表/mnist-subgraph.png)

### 推理（Inference）

inference()函数会尽肯能地构建计算图，做到返回包含了预测结果的张量。

它接受图片占位符为输入，在此基础上借助ReLU激活函数构建一对完全连接层以及一个有着十个节点、指明了输出Logits模型的线性层。

每一层都创建于一个唯一的tf.name_scope之下，创建于该作用域之下的所有元素都将带有其前缀。

    with tf.name_scope("hidden1") as scope:

在定义的作用域中，每一层所使用的权重和偏置都在tf.Variable实例中生成，并且包含了各自期望的shape。

    weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))), name="weights")
    biases = tf.Variable(tf.zeros([hidden1_units]), name="biases")

例如，当这些层是在hidden1作用域下生成时，赋予权重变量的独特名字将会是“hidden1/weights”、“hidden1/biases”。

每个变量在构建时，都会获得初始化操作。

这种最常见的情况下，通过tf.truncated_normal函数初始化权重变量，给赋予的shape则是一个二维的张量，其中第一个维度代表该层权重所连接的单元数量，第二个维度代表该层权重变量所连接**到**的单元数量。对于名叫hidden1的层，相应的维度则是[IMAGE_PIXELS, hidden1_units]，因为权重变量将图片输入连接到hidden1层。tf.truncated_normal初始化函数将根据所得到的均值和标准差生成一个随机分布（截尾正态）。然后通过tf.zeros函数初始化偏置，确保所有偏置起初都是零，而其shape则是其在该层中所连接**到**的单元数量。

计算图中的三个主要操作，分别是两个tf.nn.relu操作中嵌入的隐藏层所需的tf.matmul、Logits模型所需的另外一个tf.matmul。三者依次生成，各自的tf.Variable实例则与输入占位符或下一层的输出张量连接。

    hidden1 = tf.nn.relu(tf.matmul(imags, weights) + biases)
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    logits = tf.matmul(hidden2, weights) + biases

最后程序返回包含了输出结果的Logits模型张量。

### 损失（Loss）

loss()函数通过添加所需的损失操作，进一步构建计算图。

首先，labels_placeholder中的值将被编码为一个One-Hot的张量。例如，如果类标识符为“3”，则将转换为“[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]”。

    batch_size = tf.size(labels)
    labels = tf.expand_dims(labels, 1)
    indices = tf.expand_dims(tf.range(0, batch_size - 1), 1)
    concated = tf.concat(1, [indices, labels])
    onehot_labels = tf.sparse_to_dense(concated, tf.pack([batch_size, NUM_CLASSES]), 1.0, 0.0) # stack instead of pack

之后，又添加一个tf.nn.softmax_cross_entropy_with_logits操作，用来比较inference()函数和One-Hot标签所输出的Logits张量。

    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, onehot_labels, name="xentropy")

接着，使用tf.reduce_mean函数计算batch维度（第一维）下交叉熵的平均值，将该值作为总损失。

    loss = tf.reduce_mean(cross_entropy, name="xentropy_mean")

最后，程序返回包含了损失的张量。

**交叉熵是信息论中的概念，可以描述（神经网络）基于已有事实所做的推测最坏会导致什么结果**。

### 训练

training()函数添加通过梯度下降将损失最小化所需的操作。

首先，该从loss()函数中获取损失张量，将其交给tf.scalar_summary，后者在与SummaryWriter配合使用时，可以向事件文件中生成汇总值。本文中，每次写入汇总值时，它都会释放损失张量的当前值。

    tf.scalar_summary(loss.op.name, loss)

接下来，实例化一个tf.train.GradientDescentOptimizer，负责按照所要求的学习率应用梯度下降法。

    optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)

之后，生成一个变量用于保存全局悬链步骤的数值，并使用minimize()函数更新系统中的三角权重（Triangle Weights）、增加全局步骤的操作。根据惯例，这个操作被称为train_op，是TF会话秀发的一个完整训练步骤所必须的操作。

    global_step = tf.Variable(0, name="global_step", trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)

最后，程序返回包含了训练操作输出结果的张量。

## 训练模型

一旦计算图构建完毕，就通过fully_connected_feed_py文件中的用户代码进行迭代训练和评估。

### 计算图

在run_training()函数伊始，是一个Python语言中的with命令，这个命令表明所有已经构建的操作都要与默认的tf.Graph全局实例关联起来。

    with tf.Graph().as_default():

其中tf.Graph实例是一系列可作为整体执行的操作。TF的大部分场景只需要以来默认的计算图。

### 会话

完成全部的构建准备、生成全部所需的操作之后，就可以创建一个Session用于运行计算图。

    sess = tf.Session()

也可以利用with代码块，限制作用域。

    with tf.Session() as sess:

Session()函数中没有传入参数，表明该代码会依附于默认的本地会话（若没有创建会话则创建新的会话）。

声称会话之后，所有tf.Variable实例都会立即通过调用各自初始化操作中的sess.run()函数进行初始化。

    init = tf.initialize_all_variables()
    sess.run(init)

注意：sess.run()方法会运行计算图中**和作为参数传入的操作相对应的完整子集**，在初次调用时，init操作只包含了变量初始化程序tf.group，计算图的其余部分不会在这里，而是在下面的迭代训练中进行。

### 迭代训练

完成会话中变量的初始化就可以启动训练了。

训练的每一步都是通过客户代码控制，而能实现有效训练的最简单循环就是：

    for step in range(max_steps):
      sess.run(train_op)

但是，本文的例子要更复杂些，原因是必须把输入的数据根据每一步的情况进行切分，以匹配之前声称的占位符。

#### 向计算图提供反馈

执行每一步时，客户代码会生成一个反馈字典，其中包含对应步骤中训练所需的用例，这些用例的哈希键就是其所代表的占位符操作。

fill_feed_dict函数会查询给定的DataSet，索要下一批次batch_size的图片和标签，与占位符相匹配的张量则会包含下一批次的图片和标签。

    images_feed, lables_feed = data_set.next_batch(FLAGS.batch_size)

然后，以占位符作为哈希键，创建一个Python字典对象，值则是键代表的反馈张量。

    feed_dict = {images_placeholder: images_feed, labels_placeholder: labels_feed}

这个字典随后作为feed_dict参数，传入sess.run()函数，为这一步的训练提供输入样例。

#### 检查状态

在运行sess.run函数时，要在代码中明确其需要获取的两个值：train_op和loss。

    for step in range(FLAGS.max_steps):
      feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)
      _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)

因为需要获取这两个值，sess.run()会返回一个有两个元素的元组，其中每一个张量对象，对应了返回的元组中的NumPy数组，而这些数组中包含了当前这步训练中对应的张量。由于train_op并不会产生输出，其在返回的元组中的对应元素就是None，所以会被抛弃。但是，如果模型在训练中出现偏置，张量loss可能会变成NaN，所以需要获取它并记录下来。

假设训练一切正常，没有出现NaN,训练循环会每隔100个训练步骤打印遗憾一行简单的状态文本，告知用户当前的训练状态。

    if step%100 == 0:
      print("step", step, "loss = ", loss_value, duration, "seconds")

#### 状态可视化

为了释放[TensorBoard](./TensorBoard.ipynb)所使用的事件文件，所有的即时数据（这里只有一个）都需要在计算图构建阶段合并到一个操作中。

    summary_op = tf.merge_all_summaries()

在创建好会话之后，可以实例化一个tf.train.SummaryWriter用于写入包含计算图本身和即时数据具体值的事件文件。

    summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, graph_def=sess.graph_def)

最后，每次运行summary_op时，都会往事件文件中写入最新的即时数据，函数的输出会传入事件文件读写器的add_summary函数。

    summary_str = sess.run(summary_op, feed_dict=feed_dict)
    summary_writer.add_summary(summary_str, step)

事件文件写入完毕后，可以**就**训练文件夹打开一个TensorBoard查看即时数据的情况。

![](./图表/mnist-tensorboard.png)

了解更多构建和运行TensorBoard的信息，参考[TensorBoard](./TensorBoard.ipynb)。

#### 保存检查点

为了得到可以用于后续回复模型、以进一步训练或评估的检查点文件，需要实例化一个tf.train.Saver。

    saver = tf.train.Saver()

在训练迭代中，顶起调用saver.save()方法，向训练文件夹写入包含了当前所有可训练变量值的检查点文件。

    saver.save(sess, FLAGS.train_dir, global_step=step)

这样，以后就可以用saver.restore()方法，重载模型的参数，继续训练。

    saver.restore(sess, FLAGS.train_dir)

### 评估模型

每1000次训练迭代，客户代码尝试使用训练数据集合测试数据集对模型进行评估。do_eval函数会被调用三次，分别使用训练数据集、验证数据集合测试数据集。

    print("training data eval")
    do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.train)
    print("valicaton data eval")
    do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.validation)
    print("test data eval")
    do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.test)

注意：更复杂的应用场景通常是，先隔绝data_sets.test测试数据集，只在大量的超参数优化调整（Hyperparameter Tuning）之后才进行检查。但是，由于MNIST问题比较简单，这里一次性评估所有的数据。

#### 构建评估计算图

在打开默认的计算图之前，应该先调用get_data(train=False)函数抓取测试数据集。

    test_all_images, test_all_labels = get_data(train=False)

在进入训练迭代之前，应该先调用mnist.py文件中的evaluation函数，传入的Logits模型和标签参数要与loss函数的一致。这样做是为了先构建Eval操作。

    eval_correct = mnist.evaluation(logits, labels_placeholder)

evaluation函数会生成tf.nn.in_top_k操作，如果在K个最有可能的预测中可以发现真的标签，那么这个操作就会将模型输出标记为正确。在本文中，把K的值设置为1，也就是只有在预测是真的标签时才判定是正确的。

    eval_correct = tf.nn.in_top_k(logits, labels, 1)

#### 评估计算图的输出

之后，可以创建一个循环，往其中添加feed_dict，并在调用sess.run函数时传入eval_correct操作，目的就是用给定的数据集评估模型。

    for step in range(steps_per_epoch):
      feed_dict = fill_feed_dict(data_set, images_placeholder, labels_placeholder)
      true_count += sess.run(eval_correct, feed_dict=feed_dict)

true_count变量会累计所有in_top_k操作判定为正确的预测。接下来，只需要将正确的预测总是除以用例总数，就可以得到准确率。

    precision = float(true_count) / float(num_examples)
    print("examples", num_examples, "correct", true_count, "precision", precision)